# MUSIC GENERATION USING RNN

#### Initialization of helper functions and libraries

In [1]:
# Let's start with some packages we need
from __future__ import print_function
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys

#### Sentence Tokenization and indexing

In [2]:
def sentenceToTensor(tokens):
    # Convert list of strings to tensor of token indices (integers)
    #
    # Input
    #  tokens_list : list of strings, e.g. ['<SOS>','lion','eat','man','<EOS>']
    # Output
    #  1D tensor of the same length (integers), e.g., tensor([ 2, 18, 13, 19,  0])
    out = torch.zeros(len(tokens)).long()
    for i, token in enumerate(tokens):
        out[i] = char_idx.index(token)
    return out

# load and process the set of simple sentences
with open('data/input.txt','r') as fid:
    lines = fid.readlines()

cur_line = ''
input_lines = []
for line in lines:
    if line == '\n':
        input_lines.append(cur_line)
        cur_line = ''
        continue
    cur_line += line

input_lines = [s for s in input_lines if len(s)>0]
input_lines = ['<start>\n'+s+'<end>\n' for s in input_lines]

char_idx = ''.join(set(list((open('data/input.txt','r').read()) + '<start>' + '<end>')))
unique_tokens = list(char_idx)
    
n_tokens = len(unique_tokens) # all words and special tokens

token_to_index = {t : i for i,t in enumerate(unique_tokens)}
index_to_token = {i : t for i,t in enumerate(unique_tokens)}

training_pats = [sentenceToTensor(s) for s in input_lines]
training_pats = [s for s in training_pats if s.size()[0]>0]

ntrain = len(training_pats)
print('mapping unique tokens to integers: %s \n' % token_to_index)
print('example sentence as string: %s \n' % ''.join(input_lines[0]))
print('example sentence as tensor: %s \n' % training_pats[0])


mapping unique tokens to integers: {'k': 0, 'v': 1, '4': 2, '=': 3, 'J': 4, '~': 5, 'l': 6, 't': 7, 'e': 8, '0': 9, 'D': 10, '6': 11, 'x': 12, 'Q': 13, '?': 14, '7': 15, 'B': 16, 'M': 17, '8': 18, 'h': 19, 'm': 20, 'L': 21, 'H': 22, ']': 23, '>': 24, '%': 25, 'g': 26, 'A': 27, 'f': 28, 'b': 29, 'c': 30, ' ': 31, 'r': 32, '&': 33, 'W': 34, '2': 35, 'F': 36, 'Y': 37, 'j': 38, '_': 39, '!': 40, 'S': 41, '+': 42, '1': 43, 'o': 44, '\\': 45, '-': 46, '9': 47, 'p': 48, 'z': 49, 'n': 50, '(': 51, '/': 52, '5': 53, 'X': 54, 'N': 55, 'q': 56, 'U': 57, 'T': 58, 'y': 59, '3': 60, 's': 61, "'": 62, '^': 63, 'a': 64, 'K': 65, 'i': 66, '#': 67, '\n': 68, 'u': 69, '[': 70, 'E': 71, ',': 72, '|': 73, ':': 74, 'R': 75, 'd': 76, '.': 77, 'G': 78, 'P': 79, '<': 80, 'V': 81, 'w': 82, 'I': 83, '"': 84, 'C': 85, 'O': 86, ')': 87} 

example sentence as string: <start>
X: 1
T:A and D
% Nottingham Music Database
S:EF
Y:AB
M:4/4
K:A
M:6/8
P:A
f|"A"ecc c2f|"A"ecc c2f|"A"ecc c2f|"Bm"BcB "E7"B2f|
"A"ecc c2f|"A"ecc

### Music Recurrent Network

#### RNN Network Architecture

In [4]:
class MusicGRU(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.word_embeddings = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, num_layers)
        self.fc1 = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input_token):
        input_embed = self.word_embeddings(input_token)
        input_embed = input_embed.view(1, 1, -1)
        rnn_out, self.hidden = self.rnn(input_embed, self.hidden)
        output = self.fc1(rnn_out.view(1, -1))
        return output

    def initHidden(self):
        # Returns length hidden_size 1D tensor of zeros
        self.hidden = torch.zeros(self.num_layers, 1, self.hidden_size)

#### Training Loop and Loss Calculation

In [5]:
def train(seq_tensor, rnn):
    target = seq_tensor
    target = torch.cat([target[1:]])
    outputs = []
    rnn.initHidden()
    optimizer.zero_grad()
    for i in range(len(seq_tensor)-1):
        output = rnn(seq_tensor[i])
        output = torch.flatten(output)
        outputs.append(output)
    outputs = torch.stack(outputs)
    loss = criterion(outputs, target)
    loss.backward()
    return loss

#### Parameter Initialization, Main Code and Checkpointing 

In [ ]:
nepochs = 100 #number of passes through the entire training set
in_size = n_tokens
out_size = n_tokens
num_layers, HIDDEN_SIZE = 1, 150
model_type = 'gru'
CHECKPOINT = 'ckpt_mdl_{}_hsize_{}'.format(model_type, HIDDEN_SIZE)

losses = []
start_epoch = 1
RESUME = False
print("Running.............")

if RESUME:
    # Load checkpoint.
    print('==> Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    
    assert os.path.isdir('epochs'), 'Error: no last epoch tracing directory found!'
    
    f = open('./epochs/epochtrace.txt', "r")
    lp = f.read()
    
    last_epoch = lp.strip()
    
    checkpoint = torch.load('./checkpoint/' + CHECKPOINT + '.t' + last_epoch)
    rnn = checkpoint['mode']
    loss = checkpoint['loss']
    losses = checkpoint['losses']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building model...')
    rnn = MusicGRU(n_tokens, HIDDEN_SIZE, out_size, num_layers)

optimizer = torch.optim.AdamW(rnn.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(start_epoch, nepochs + 1):
    loss = 0
    perm = np.random.permutation(len(training_pats))
    for p in perm: # visit data points in random order
        x_pat = training_pats[p] # get one input pattern
        cur_loss = train(x_pat, rnn)
        loss += cur_loss.item()
        optimizer.step()
    losses.append(loss/len(training_pats))
    print("epoch = ", epoch, "Avg Loss = ", loss/len(training_pats))
    
    state = {
            'mode': rnn,
            'loss': losses[-1],
            'losses': losses,
            'epoch': epoch,
    }
    
    last_epoch = str(epoch)
    
    if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + CHECKPOINT + '.t%s' % epoch)
    
    if not os.path.isdir('epochs'):
            os.mkdir('epochs')
    f = open('./epochs/epochtrace.txt', "w")
    f.write(last_epoch)
    f.close()

Running.............
==> Building model...


#### Music Generator and Tester

In [ ]:
def generate_song(prime_str='<start>', max_len=1000, temp=0.8):
    with torch.no_grad():
        rnn.initHidden()
        creation = '<start>'
        prime = sentenceToTensor(creation)
        for i in range(len(prime)-1):
            _ = rnn(prime[i])

        # Generate rest of sequence
        for j in range(max_len):
            out = rnn(sentenceToTensor(creation[-1])).data.view(-1)

            out = np.array(np.exp(out/temp))
            dist = out / np.sum(out)np

            # Add predicted character to string and use as next input        
            creation += char_idx[np.random.choice(len(dist), p=dist)]
            if creation[-5:] == '<end>':
                break
        print(creation)

In [ ]:
generate_song(max_len=1000, temp=0.8)

Listen your generated music here: https://abc.rectanglered.com/

In [72]:
# from torch.distributions.categorical import Categorical
# def generate(rnn, maxlen=1000):
#      # TODO : YOUR CODE GOES HERE
#      with torch.no_grad():
#          input = torch.tensor(28)
#          outputs = []
#          outputs.append(index_to_token[input.item()])
#          rnn.initHidden()
#          for i in range(maxlen):
#              output = rnn(input)
#              m = Categorical(torch.exp(output))
#              input = m.sample()
#              word = index_to_token[input.item()]
#              outputs.append(word)
#          st = ""
#          for i in range(len(outputs)):
#              st = st + outputs[i] + "";
#          print(st)

# for i in range(1):
#  generate(rnn)